In [1]:
import scipy.io

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as grd

import numpy as np

from game import Game
from strategies import FastAttractorMemory, BaseStrategy, PerfectMemory
from memory import FastAttractor

import torch
from tqdm import tqdm

from fiftyone import ViewField as F

import scipy.io as sio

import math

In [ ]:
n_trials = 20

steps = 10

size_range = [[3,3], [4,4], [5,5], [6,6], [7,7]]

: 

In [ ]:
def save_mat(path, df):
    result = []

    for column in df.columns:
        entries = df[column].values
        column_tuple = (np.array([column]), entries.reshape(-1, 1))
        result.append(column_tuple)

    # Convert the result list to an array
    result_array = np.array(result)

    sio.savemat(path, {'b_data': result_array})

: 

In [ ]:
def play_strategy(strategy, game, max_steps=1500):
    win = False
    steps = 0
    while not win and steps < max_steps:
        strategy.pick()
        win = game.check_win()
        torch.cuda.empty_cache()
        steps+=1

: 

In [ ]:
def collect_data(n_trials, size_range, strategy, game):
    dfs = []
    tot_clicks = []
    for i, size in enumerate(size_range): 
        size_clicks = []
        for n in tqdm(range(n_trials), desc=f'Size: {size}'):
            game.set_size(size, 2)
            strategy.reset()
            play_strategy(strategy, game)
            df, nc = game.get_metrics()
            df['id'] = i*n_trials + n
            dfs.append(df)
            size_clicks.append(nc)
        tot_clicks.append(size_clicks)
    return pd.concat(dfs).reset_index(drop=True), tot_clicks

: 

In [ ]:
def add_reaction_times_confidence(activation_history, df, size):
    tiles_clicked = df['tile_clicked'].to_numpy()-1
    indices = np.tile(np.repeat([False, True, False], steps), len(tiles_clicked) // 2)
    history = activation_history[indices]
    neuron = np.repeat(tiles_clicked[1::2], steps)
    x = history[torch.arange(history.size(0)), neuron].view(-1, steps)
    max_v, _ = torch.max(x, dim=1, keepdim=True)
    means = torch.mean(history[:, :size], axis=1).numpy()
    df['reaction_time'] = np.repeat(np.argmax(torch.abs(max_v - x) <= torch.abs(0.1*max_v), axis=1), 2)
    df['confidence'] = np.repeat(torch.abs(max_v.flatten() - means[steps-1::steps]) / torch.max(torch.abs(max_v.flatten() - means[steps-1::steps])), 2) # TODO check if it changes if mean is without max neuron
    return df
# def add_reaction_times_confidence(activation_history, df, size):
#     tiles_clicked = df['tile_clicked'].to_numpy()
#     indices = np.tile(np.repeat([False, True, False], steps), len(tiles_clicked) // 2)
#     history = activation_history[indices]
#     neuron = np.repeat(tiles_clicked[1::2], steps)
#     x = history[torch.arange(history.size(0)), neuron].view(-1, steps)
#     max_v, _ = torch.max(x, dim=1, keepdim=True)
#     means = torch.mean(history[:, :size], axis=1).numpy().reshape(-1, steps)
#     df['reaction_time'] = np.repeat(np.argmax(torch.abs(max_v - x) <= torch.abs(0.1*max_v), axis=1), 2)
#     df['confidence'] = np.repeat(torch.abs(torch.repeat_interleave(max_v, steps, dim=1) - means), 2, axis=0).tolist() # TODO check if it changes if mean is without max neuron
#     return df

: 

In [ ]:
def extract_energy(energy_history, df):
    tiles_clicked = df['tile_clicked'].to_numpy()-1
    indices = np.tile(np.repeat([True, False, False], steps), len(tiles_clicked) // 2)
    energy = energy_history[indices].view(-1, steps)
    max_v, _ = torch.max(energy, dim=1, keepdim=True)
    min_v, _ = torch.min(energy, dim=1, keepdim=True)
    df['max_energy'] = np.repeat((max_v.flatten() - min_v.flatten()) / -min_v.flatten(), 2)
    return df
# def extract_energy(energy_history, df):
#     tiles_clicked = df['tile_clicked'].to_numpy()
#     indices = np.tile(np.repeat([True, False, False], steps), len(tiles_clicked) // 2)
#     energy = energy_history[indices].view(-1, steps)
#     df['max_energy'] = np.repeat(energy, 2, axis=0).tolist()
#     return df

: 

In [ ]:
def collect_data_attractor(n_trials, size_range, game, lr, rr):
    dfs = []
    tot_clicks = []
    for i, size in enumerate(size_range): 
        size_clicks = []
        for n in tqdm(range(n_trials), desc=f'Size: {size}'):
            game.set_size(size, 2)
            dim = np.prod(size) + len(np.unique(game.get_grid_labels()))
            memory = FastAttractor(dim, lr, rr)
            strategy = FastAttractorMemory(game, memory, steps=steps)
            play_strategy(strategy, game)
            history = memory.get_activation_history()
            df, nc = game.get_metrics()
            df = add_reaction_times_confidence(history, df, np.prod(size))
            energy_history = memory.get_energy_history()
            df = extract_energy(energy_history, df)
            df['id'] = i*n_trials + n
            dfs.append(df)
            size_clicks.append(nc)
        tot_clicks.append(size_clicks)
    return pd.concat(dfs).reset_index(drop=True), tot_clicks

: 

In [ ]:
def get_second_click_df(df):
    o_df = pd.DataFrame()
    o_df['id'] = df['id'].to_numpy()[1::2]
    o_df['match'] = df['match'].to_numpy()[1::2]
    o_df['random_match'] = df['random_match'].to_numpy()[1::2]
    o_df['board_size'] = df['board_size'].to_numpy()[1::2]
    o_df['tile_clicked'] = df['tile_clicked'].to_numpy()[1::2]
    o_df['nslc'] = df['nslc'].to_numpy()[1::2]
    o_df['nslc_first'] = df['nslc'].to_numpy()[0::2]
    o_df['nsp'] = df['nsp'].to_numpy()[1::2]
    o_df['correct_tile'] = df['correct_tile'].to_numpy()[0::2]
    # o_df['reaction_time'] = df['reaction_time'].to_numpy()[1::2]
    # o_df['confidence'] = df['confidence'].to_numpy()[1::2]
    # o_df['max_energy'] = df['max_energy'].to_numpy()[1::2]
    return o_df

: 

In [ ]:
def build_nc_df(nc, strategy):
    data = []
    for i, size in enumerate(size_range):
        for j in range(n_trials):
            data.append({'size': str(size[0]), 'nc': math.log10(nc[i][j]/np.prod(size))})
    data = pd.DataFrame(data)
    data['strategy'] = strategy
    return data

: 

In [ ]:
game = Game(
            dataset_name='coco-2017', 
            split='validation', 
            field="ground_truth", 
            ds_filter=F("supercategory").is_in(('person', 'animal', 'food', 'vehicle', 'indoor')),
            dataset_dir ="/home/ravi/datasets/coco-2017",
        )

: 

# Benchmarks

In [ ]:
bs = BaseStrategy(game)
bs_df, bs_nc = collect_data(n_trials, size_range, bs, game)
bs_df = get_second_click_df(bs_df)
save_mat('random_memory_metrics.mat', bs_df)
bs_nc_df = build_nc_df(bs_nc, 'Random Memory')
bs_df['strategy'] = 'Random memory'

: 

In [ ]:
pm = PerfectMemory(game)
pm_df, pm_nc = collect_data(n_trials, size_range, pm, game)
pm_df = get_second_click_df(pm_df)
save_mat('perfect_memory_metrics.mat', pm_df)
pm_nc_df = build_nc_df(pm_nc, 'Perfect Memory')
pm_df['strategy'] = 'Perfect memory'

: 

# Attractor

In [ ]:
att_df_double, att_nc = collect_data_attractor(n_trials, size_range, game, .7, .9)
att_df = get_second_click_df(att_df_double)
save_mat('model_metrics.mat', att_df)
att_nc_df = build_nc_df(att_nc, 'Attractor Memory')
att_df['strategy'] = 'Attractor Memory'

: 

# Behavioral metrics

In [ ]:
df = att_df
df['Trial'] = 'Mismatch'
df.loc[df['match'] == 1, 'Trial'] = 'Match'
df['board_size'] = df['board_size'].apply(np.sqrt)

: 

## Reaction Time

In [ ]:
plt.figure(figsize=(8, 16)) # TODO check when the neuron converges
palette = ['green', 'grey'] 
# Draw a nested barplot by species and sex
ax = sns.catplot(x="board_size", y="reaction_time", hue='Trial', data=df, aspect=0.85, kind='bar', errorbar='se', palette=palette, hue_order=['Match', 'Mismatch'])
ax.set(xlabel="Board size", ylabel="reaction-time")
plt.show()

: 

# Confidence

In [ ]:
plt.figure(figsize=(8, 16))
palette = ['green', 'grey']
# Draw a nested barplot by species and sex
ax = sns.catplot(x="board_size", y="confidence", hue='Trial', data=df, aspect=0.85, kind='bar', errorbar='se', palette=palette, hue_order=['Match', 'Mismatch'])
ax.set(xlabel="Board size", ylabel="reaction-confidence")
plt.show()

# b_grouped = df.groupby('board_size')
# for board_size, b_group in b_grouped:
#     palette = ['blue', 'yellow', 'orange']
#     plt.figure(figsize=(16, 16))
#     # ax = sns.lineplot(y="max_energy", data=b_group, aspect=0.85, palette=palette)
#     # ax.set(xlabel="Board size", ylabel="surprise-response")
#     df_plot = b_group.explode('max_energy').reset_index(drop=True)
#     df_plot['index'] = df_plot.index % 10
#     sns.lineplot(data=df_plot, x='index', y='max_energy', hue='Trial')

: 

# Surprise

In [ ]:
n_df = pd.DataFrame()
n_df['nslc'] = att_df_double['nslc'].to_numpy()[0::2]
n_df['max_energy'] = att_df_double['max_energy'].to_numpy()[0::2]
n_df['board_size'] = att_df_double['board_size'].to_numpy()[0::2]

nslc_first = n_df[n_df['nslc'] <= 0]
nslc_first['Data'] = 'First-click'

nslc_old = n_df[n_df['nslc'] > 1]
nslc_old['Data'] = 'nslc>1'

# nslc_older = n_df[n_df['nslc'] >= 5]
# nslc_older['Data'] = 'nslc>5'

nslc_recent = n_df[n_df['nslc'] == 1]
nslc_recent['Data'] = 'nslc=1'

nslc_df = pd.concat([nslc_first, nslc_old, nslc_recent])

# b_grouped = nslc_df.groupby('board_size')
# for board_size, b_group in b_grouped:
#     palette = ['blue', 'yellow', 'orange']
#     plt.figure(figsize=(16, 16))
#     # ax = sns.lineplot(y="max_energy", data=b_group, aspect=0.85, palette=palette)
#     # ax.set(xlabel="Board size", ylabel="surprise-response")
#     df_plot = b_group.explode('max_energy').reset_index(drop=True)
#     df_plot['index'] = df_plot.index % 10
#     sns.lineplot(data=df_plot, x='index', y='max_energy', hue='Data')

b_grouped = nslc_df.groupby('board_size')
for board_size, b_group in b_grouped:
    palette = ['blue', 'yellow', 'orange']
    plt.figure(figsize=(16, 16))
    ax = sns.catplot(x="board_size", y="max_energy", hue='Data', data=b_group, aspect=0.85, kind='bar', palette=palette)
    ax.set(xlabel="Board size", ylabel="surprise-response")
    plt.show()

# palette = ['blue', 'yellow', 'orange']
# plt.figure(figsize=(16, 16))
# ax = sns.catplot(x="board_size", y="max_energy", hue='Data', data=nslc_df, aspect=0.85, kind='bar', palette=palette)
# ax.set(xlabel="Board size", ylabel="surprise-response")
# plt.show()

: 

## nsp

In [ ]:
plt.figure(figsize=(8, 16))
palette = ['green', 'grey']
# Draw a nested barplot by species and sex
ax = sns.catplot(x="board_size", y="nsp", hue='Trial', data=df[df['nsp'] > 0], aspect=0.85, errorbar='se', kind='bar', palette=palette, hue_order=['Match', 'Mismatch'])
ax.set(xlabel="Board size", ylabel="n-since-pair")
plt.show()

: 

## NSLC

In [ ]:
plt.figure(figsize=(8, 16))
palette = ['green', 'grey']
# Draw a nested barplot by species and sex
ax = sns.catplot(x="board_size", y="nslc", hue='Trial', data=df[df['nslc'] > 0], aspect=0.85, errorbar='se', kind='bar', palette=palette, hue_order=['Match', 'Mismatch'])
ax.set(xlabel="Board size", ylabel="n-since-last-click")
plt.show()

: 

## Tot clicks

In [ ]:
data = pd.concat([bs_nc_df, pm_nc_df, att_nc_df])
plt.figure(figsize=(8, 16))
palette = ['red', 'blue', 'plum']
ax2 = sns.catplot(x="size", y='nc', hue="strategy", data=data, aspect=0.85, errorbar=None, kind='point', palette=palette)
ax = sns.scatterplot(x="size", y='nc', hue="strategy", data=data, legend=False, palette=palette)
ax.set_xlabel("Board size")
ax.set_ylabel("N clicks per tile (log10)")
sns.move_legend(ax2, "upper center")

: 

# Memory history

In [ ]:
def plot_history(locations, category):
    # create a 3 X 2 grid 
    fig = plt.figure(figsize=(24, 10))
    gs = grd.GridSpec(3, 2, figure=fig, height_ratios=[len(locations),len(locations),len(category)], width_ratios=[18,1], wspace=0.4)
    # image plot
    ax1 = plt.subplot(gs[1,0])
    p1 = ax1.imshow(locations, aspect='auto', interpolation='none', cmap='hot')
    colorAx1 = plt.subplot(gs[1,1])
    cb1 = plt.colorbar(p1, cax = colorAx1)
    # image plot
    ax2 = plt.subplot(gs[2,0])
    p2 = ax2.imshow(category, aspect='auto', interpolation='none', cmap='hot')
    colorAx2 = plt.subplot(gs[2,1])
    cb2 = plt.colorbar(p2, cax = colorAx2)
    
    plt.show()

: 

In [ ]:
size = [3,3]
game.set_size(size, 2)
pos_f = np.prod(size)
lab_f = len(np.unique(game.get_grid_labels()))
dim = pos_f + lab_f
memory = FastAttractor(dim, lr=0.5, rr=0.95)
strategy = FastAttractorMemory(game, memory, steps=steps)
print(game.get_grid_labels())
print(np.arange(9))
tot = play_strategy(strategy, game)
print(tot)
ah = memory.get_activation_history()
plot_history(ah[:, :pos_f].T, ah[:, pos_f:].T)

: 

In [ ]:
plt.plot(memory.get_energy_history())

: 

In [ ]:
df, tot_clicks = game.get_metrics()
df['nslc'].to_numpy()
events_dict = {'probe': 'blue', 'response': 'red', 'delay': 'pink', 'stimulus': 'green'}
plot_history(f.T[:9], f.T[10:19], c, strategy.t_steps, strategy.events, events_dict)

: 